In [3]:
%use adventOfCode

In [4]:
val aoc = AocClient.fromEnv().interactiveDay(2025, 4)
aoc.viewPartOne()

--- Day 4: Printing Department --- 
 You ride the escalator down to the printing department. They're clearly getting ready for Christmas; they have lots of large rolls of paper everywhere, and there's even a massive printer in the corner (to handle the really big print jobs). 
 Decorating here will be easy: they can make their own decorations. What you really need is a way to get further into the North Pole base while the elevators are offline. 
 "Actually, maybe we can help with that," one of the Elves replies when you ask for help. "We're pretty sure there's a cafeteria on the other side of the back wall. If we could break through the wall, you'd be able to keep moving. It's too bad all of our forklifts are so busy moving those big rolls of paper around." 
 If you can optimize the work the forklifts are doing, maybe they would have time to spare to break through the wall. 
 The rolls of paper ( @ ) are arranged on a large grid; the Elves even have a helpful diagram (your puzzle input) indicating where everything is located. 
 For example: 
 ..@@.@@@@.
@@@.@.@.@@
@@@@@.@.@@
@.@@@@..@.
@@.@@@@.@@
.@@@@@@@.@
.@.@.@.@@@
@.@@@.@@@@
.@@@@@@@@.
@.@.@@@.@.
 
 The forklifts can only access a roll of paper if there are fewer than four rolls of paper in the eight adjacent positions. If you can figure out which rolls of paper the forklifts can access, they'll spend less time looking and more time breaking down the wall to the cafeteria. 
 In this example, there are 13 rolls of paper that can be accessed by a forklift (marked with x ): 
 ..xx.xx@x.
x@@.@.@.@@
@@@@@.x.@@
@.@@@@..@.
x@.@@@@.@x
.@@@@@@@.@
.@.@.@.@@@
x.@@@.@@@@
.@@@@@@@@.
x.x.@@@.x.
 
 Consider your complete diagram of the paper roll locations. How many rolls of paper can be accessed by a forklift?

In [6]:
val exampleInput = """
..@@.@@@@.
@@@.@.@.@@
@@@@@.@.@@
@.@@@@..@.
@@.@@@@.@@
.@@@@@@@.@
.@.@.@.@@@
@.@@@.@@@@
.@@@@@@@@.
@.@.@@@.@.
""".trimIndent()

In [66]:
data class Roll(val x: Int, val y: Int)
class Grid(private val grid: List<MutableList<Boolean>>) {

    fun getRollOrNull(x: Int, y: Int): Roll? = grid.getOrNull(y)?.getOrNull(x)?.let {
        if (grid[y][x]) Roll(x, y) else null
    }

    fun neighboursOf(roll: Roll): List<Roll> = neighboursOf(roll.x, roll.y)
    fun neighboursOf(x: Int, y: Int): List<Roll> =
        listOfNotNull(
            getRollOrNull(x, y - 1),
            getRollOrNull(x, y + 1),
            getRollOrNull(x - 1, y - 1),
            getRollOrNull(x - 1, y),
            getRollOrNull(x - 1, y + 1),
            getRollOrNull(x + 1, y - 1),
            getRollOrNull(x + 1, y),
            getRollOrNull(x + 1, y + 1),
        )

    val rolls: List<Roll>
        get() = grid.indices.flatMap { y ->
            grid[y].indices.mapNotNull { x ->
                getRollOrNull(x, y)
            }
        }

    val accessibleRolls: List<Roll>
        get() = rolls.filter {
            neighboursOf(it).size < 4
        }

    override fun toString(): String =
        grid.joinToString("\n") {
            it.joinToString("") {
                if (it) "@" else "."
            }
        }

    fun printAccessible() =
        grid.withIndex().joinToString("\n") { (y, row) ->
            row.withIndex().joinToString("") { (x, it) ->
                when (it) {
                    true -> {
                        if (neighboursOf(x, y).size < 4) "x" else "@"
                    }

                    else -> "."
                }
            }
        }

    // for pt2
    fun removeAll(rolls: Iterable<Roll>) {
        rolls.forEach(::remove)
    }

    fun remove(roll: Roll) {
        grid[roll.y][roll.x] = false
    }
}

fun String.parse(): Grid = Grid(trimIndent().lines().map { it.map { it == '@' }.toMutableList() })

In [67]:
val exampleGrid = exampleInput.parse()
exampleGrid

..@@.@@@@.
@@@.@.@.@@
@@@@@.@.@@
@.@@@@..@.
@@.@@@@.@@
.@@@@@@@.@
.@.@.@.@@@
@.@@@.@@@@
.@@@@@@@@.
@.@.@@@.@.

In [68]:
exampleGrid.accessibleRolls.size

13

In [51]:
exampleGrid.printAccessible()

..xx.xx@x.
x@@.@.@.@@
@@@@@.x.@@
@.@@@@..@.
x@.@@@@.@x
.@@@@@@@.@
.@.@.@.@@@
x.@@@.@@@@
.@@@@@@@@.
x.x.@@@.x.

In [69]:
val input = aoc.input().parse()
input

@@.@@..@@@@@@@.@@.@..@@..@..@@.@@@@.@@@@@.@.@..@..@@@@.@@@@..@@.@@@@.@@....@.@.@@.@.@@@@@@@@@@@@.@@..@@@..@@.@@@@@....@.@@@@@@@@..@@..@
@@@@@@@@.@....@@@@@.@.@.@@@.@@...@@@.@@.@@@@@..@@.@@@@@@.@@@.@@@.@@@.@@.@@@.@.@@.....@@.@@@@@.@.@.@...@@@@..@.@@@@@@@.@@@@@..@@.@@...@.
@.@.@.@@@@@.@@..@@@@.@@@..@@..@@@...@@.@@..@@@.@@@@.@.@@@@@@.@@@@@.@..@@@.@@@.@.@.@..@@@@.@@..@@@@......@...@@...@@.@@@.@@@@..@..@.@.@@
...@@...@@@.@@@.@@.@..@@@...@@@@...@@@@@@.@@.@@@@.@@@@@@.@..@@@@@.@@@..@@@@.@@@@@@@.@.@@@...@.@@.@@.@@@@@@.@.@@@.@@..@..@.@.@@@@@...@.@
@@.@@@@@@@.@@@..@@.@.@@@@@@.@..@.@@@@@@.@@@@@@@@..@.@@.@.@@@@@.@.@@@@.@@@@@..@..@.@.@.@@@@.@@..@....@....@.@@..@@..@@..@.@@@.@@@@@@.@..
@@@@........@@...@@@@.@@..@@@@@@..@..@@@@@@.@.@@@@@@..@@@..@@.@@@.@@@..@@@.@@.@.@@..@@.@@@.@.@..@@..@@....@..@@.@.@..@@@@@.@@..@@@..@..
@@.@@@@.@.@@@@@...@@@@@@@.@@.@@@@@@.@@@.@@@...@.@@@.@@.@@@@@.@@..@@@@@@.@@@@.@@@@@@.@.@@@@@@@..@@@@.@@.@..@@@@....@@@.@@..@@@.@..@.@@.@
.@@@@....@@@.@@@@@@..@@@..@@@@.@@@@@@@@@@.@@@.@.

In [70]:
input.printAccessible()

x@.@@..x@xxxxx.@@.x..xx..x..xx.xx@@.x@@@x.x.x..x..x@@@.x@@@..x@.x@@x.xx....x.x.xx.x.x@@@@@@@@@xx.xx..x@@..xx.x@@@@....x.@@@xx@@x..xx..x
@@@@@@@@.@....@@@@@.x.@.x@@.@@...@@@.@@.@@@@@..@@.@@@@@@.@@@.@@@.@@@.@@.x@@.@.@@.....@@.@@@@@.@.@.x...@@@x..@.@@@@@@x.@@@@@..@@.xx...x.
x.@.@.@@@@@.x@..@@@@.@@@..x@..@@x...@@.@@..@@@.@@@@.@.@@@@@@.@@@@@.@..@@@.@@@.@.@.x..@@@@.@@..@@@@......@...@@...@@.@@@.@@@@..@..x.x.@x
...@@...@@@.@@x.@@.x..@@@...x@@@...@@@@@@.@@.@@@@.@@@@@@.@..@@@@@.@@@..@@@@.@@@@@@x.x.@@@...x.@@.xx.xxxx@x.x.@@x.@@..@..@.@.@@@@@...x.x
x@.@@xxx@@.@@@..@@.@.@@@@@x.@..@.x@@@@@.@@@@@@@@..@.@@.@.x@@@@.@.@@@@.x@@@@..@..@.x.x.@@@@.x@..x....@....x.x@..@@..xx..@.@@@.@@@@@x.x..
@@@@........@@...@@@@.@@..@@@@@@..@..@@@@@@.x.@@@@@@..@@@..@@.@@@.@@@..@@@.@@.@.@@..x@.@@@.@.x..@@..@@....@..@x.x.x..@@@@@.@@..@@@..x..
@@.@@xx.x.@@@@@...@@@@@@x.@@.@@@@@@.@@@.@@@...x.@@@.@@.@@@@@.@@..@@@@@@.@@@@.@@@@@x.@.@@@@@@@..@@@@.@@.x..x@x@....@@@.@@..@@x.x..x.xx.x
.@@@@....@@@.@@@@@@..@@@..@@@@.@@@@@@@@@@.@@x.x.

In [71]:
val answer1 = input.accessibleRolls.size
answer1

1344

In [56]:
// aoc.submitPartOne(answer1)

Your answer: 1344. 
 That's the right answer! You are one gold star closer to decorating the North Pole. [Continue to Part Two]

In [57]:
aoc.viewPartTwo()

--- Part Two --- 
 Now, the Elves just need help accessing as much of the paper as they can. 
 Once a roll of paper can be accessed by a forklift, it can be removed . Once a roll of paper is removed, the forklifts might be able to access more rolls of paper, which they might also be able to remove. How many total rolls of paper could the Elves remove if they keep repeating this process? 
 Starting with the same example as above, here is one way you could remove as many rolls of paper as possible, using highlighted @ to indicate that a roll of paper is about to be removed, and using x to indicate that a roll of paper was just removed: 
 Initial state:
.. @ @ . @ @ @ @ .
 @ @@.@.@.@@
@@@@@. @ .@@
@.@@@@..@.
 @ @.@@@@.@ @ 
.@@@@@@@.@
.@.@.@.@@@
 @ .@@@.@@@@
.@@@@@@@@.
 @ . @ .@@@. @ .

Remove 13 rolls of paper:
..xx.xx @ x.
x@@. @ . @ .@ @ 
 @ @@@@.x.@@
 @ .@@@@.. @ .
x@.@@@@. @ x
. @ @@@@@@. @ 
. @ .@.@.@@@
x.@@@.@@@@
. @ @@@@@@@.
x.x.@@@.x.

Remove 12 rolls of paper:
.......x..
. @ @.x.x. @ x
x@@@@... @ @ 
x.@@@@..x.
. @ .@@@@.x.
.x@@@@@@.x
.x.@.@.@@ @ 
..@@@.@@@@
.x @ @@@@@@.
....@@@...

Remove 7 rolls of paper:
..........
.x @ .....x.
. @ @@@...xx
..@@@@....
.x.@@@@...
.. @ @@@@@..
...@.@.@@x
.. @ @@.@@@ @ 
..x@@@@@@.
....@@@...

Remove 5 rolls of paper:
..........
..x.......
.x @ @@.....
..@@@@....
...@@@@...
..x@@@@@..
...@.@.@@.
..x@@.@@@x
...@@@@@ @ .
....@@@...

Remove 2 rolls of paper:
..........
..........
..x@@.....
.. @ @@@....
...@@@@...
...@@@@@..
...@.@.@@.
...@@.@@@.
...@@@@@x.
....@@@...

Remove 1 roll of paper:
..........
..........
... @ @.....
..x@@@....
...@@@@...
...@@@@@..
...@.@.@@.
...@@.@@@.
...@@@@@..
....@@@...

Remove 1 roll of paper:
..........
..........
...x @ .....
...@@@....
...@@@@...
...@@@@@..
...@.@.@@.
...@@.@@@.
...@@@@@..
....@@@...

Remove 1 roll of paper:
..........
..........
....x.....
... @ @@....
...@@@@...
...@@@@@..
...@.@.@@.
...@@.@@@.
...@@@@@..
....@@@...

Remove 1 roll of paper:
..........
..........
..........
...x@@....
...@@@@...
...@@@@@..
...@.@.@@.
...@@.@@@.
...@@@@@..
....@@@...
 
 Stop once no more rolls of paper are accessible by a forklift. In this example, a total of 43 rolls of paper can be removed. 
 Start with your original diagram. How many rolls of paper in total can be removed by the Elves and their forklifts?

In [76]:
val exampleGrid2 = exampleInput.parse()
var removed = 0
while (true) {
    DISPLAY(exampleGrid2.printAccessible() + "\n\n")
    val rolls = exampleGrid2.accessibleRolls
    if (rolls.isEmpty()) break
    exampleGrid2.removeAll(rolls)
    removed += rolls.size
}

removed

..xx.xx@x.
x@@.@.@.@@
@@@@@.x.@@
@.@@@@..@.
x@.@@@@.@x
.@@@@@@@.@
.@.@.@.@@@
x.@@@.@@@@
.@@@@@@@@.
x.x.@@@.x.



.......x..
.@@.x.x.@x
x@@@@...@@
x.@@@@..x.
.@.@@@@.x.
.x@@@@@@.x
.x.@.@.@@@
..@@@.@@@@
.x@@@@@@@.
....@@@...



..........
.x@.....x.
.@@@@...xx
..@@@@....
.x.@@@@...
..@@@@@@..
...@.@.@@x
..@@@.@@@@
..x@@@@@@.
....@@@...



..........
..x.......
.x@@@.....
..@@@@....
...@@@@...
..x@@@@@..
...@.@.@@.
..x@@.@@@x
...@@@@@@.
....@@@...



..........
..........
..x@@.....
..@@@@....
...@@@@...
...@@@@@..
...@.@.@@.
...@@.@@@.
...@@@@@x.
....@@@...



...

..........
..........
...x@.....
...@@@....
...@@@@...
...@@@@@..
...@.@.@@.
...@@.@@@.
...@@@@@..
....@@@...



..........
..........
....x.....
...@@@....
...@@@@...
...@@@@@..
...@.@.@@.
...@@.@@@.
...@@@@@..
....@@@...



..........
..........
..........
...x@@....
...@@@@...
...@@@@@..
...@.@.@@.
...@@.@@@.
...@@@@@..
....@@@...



..........
..........
..........
....@@....
...@@@@...
...@@@@@..
...@.@.@@.
...@@.@@@.
...@@@@@..
....@@@...



43

In [77]:
val grid2 = aoc.input().parse()
var removed = 0
while (true) {
    // DISPLAY(grid2.printAccessible() + "\n\n")
    val rolls = grid2.accessibleRolls
    if (rolls.isEmpty()) break
    grid2.removeAll(rolls)
    removed += rolls.size
}

removed

8112

In [78]:
// aoc.submitPartTwo(removed)

Your answer: 8112. 
 That's the right answer! You are one gold star closer to decorating the North Pole. 
 You have completed Day 4! You can [Share on Bluesky Twitter Mastodon ] this victory or [Return to Your Advent Calendar] .